<a target="_blank" href="https://colab.research.google.com/github/Xornotor/PPGEE-Otimizacao-Exercicios/blob/main/Lista-01-A/Q8.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# **Lista de Exercícios 01-A | Questão 8**

**UFBA** | PPGEE0016 - Otimização

**Aluno:** André Paiva Conrado Rodrigues

In [1]:
# Importação de dependências
import numpy as np
from scipy.optimize import linprog
np.set_printoptions(precision=3, suppress=True)

## **1. Problema de Otimização**

Considerando:

*   $x_1$ A quantidade do Combustível A a ser fabricada;
*   $x_2$ A quantidade do Combustível B a ser fabricada;
*   $x_3$ A quantidade do Combustível C a ser fabricada;

Temos o seguinte problema de otimização:

\begin{equation*}
\begin{aligned}
\text{Maximizar} \\
&& Z = 20x_1 + 22x_2 + 18x_3 \\
\text{Sujeito a} \\
c_{1}: && 8x_1 + 5x_2 + 4x_3 \leq 120 \\
c_{2}: && 5x_1 + 4x_2 + 2x_3 \leq 200 \\
&& x_1, x_2, x_3 \geq 0 \\
\end{aligned}
\end{equation*}

## **2. Solução manual por Simplex**

Primeiramente, é necessário colocar a função objetivo e as restrições na forma padrão, adicionando variáveis de folga para eliminar as desigualdades. Deste modo, o problema é reformulado da seguinte maneira:

\begin{equation*}
\begin{aligned}
\text{Maximizar} \\
&& Z - 20x_1 - 22x_2 - 18x_3 = 0\\
\text{Sujeito a} \\
c_{1}: && 8x_1 + 5x_2 + 4x_3 + x_4 = 120 \\
c_{2}: && 5x_1 + 4x_2 + 2x_3 + x_5 = 200 \\
&& x_1, x_2, x_3, x_4, x_5 \geq 0 \\
\end{aligned}
\end{equation*}

Montando a matriz $\boldsymbol{M}$ para o algoritmo Simplex:

|        ////        | $Z$ | $x_1$ | $x_2$ | $x_3$ | $x_4$ | $x_5$ |  $b$  |
|--------------------|-----|-------|-------|-------|-------|-------|-------|
| $\boldsymbol{L_1}$ |  0  |   8   |   5   |   4   |   1   |   0   |  120  |
| $\boldsymbol{L_2}$ |  0  |   5   |   4   |   2   |   0   |   1   |  200  |
| $\boldsymbol{L_3}$ |  1  |  -20  |  -22  |  -18  |   0   |   0   |   0   |

In [2]:
M = np.array([
    [0,  8,   5,   4,  1,  0,  120 ],
    [0,  5,   4,   2,  0,  1,  200 ],
    [1, -20, -22, -18, 0,  0,  0   ]
], dtype=np.float64)

### **Iteração 1**

Iniciamos selecionando a coluna de $x_2$ como **coluna pivô** por possuir custo com menor valor (-22). A partir disto, efetuamos a divisão *element-wise* dos elementos da coluna $b$ pelos elementos da coluna $x_2$, e selecionando o menor valor, obtemos a **linha pivô**.

In [3]:
b_col = 6
pivot_col = 2
div_pivot_b_cols = (M.T[b_col] / M.T[pivot_col])[:-1]
div_pivot_b_cols

array([24., 50.])

In [4]:
pivot_row = np.argmin(div_pivot_b_cols)
pivot_row

np.int64(0)

Logo, a linha pivô é $L_1$. Fazemos, então, a substituição $L_1 \rightarrow \frac{L_1}{5}$.

In [5]:
M[pivot_row] /= M[pivot_row][pivot_col]
M

array([[  0. ,   1.6,   1. ,   0.8,   0.2,   0. ,  24. ],
       [  0. ,   5. ,   4. ,   2. ,   0. ,   1. , 200. ],
       [  1. , -20. , -22. , -18. ,   0. ,   0. ,   0. ]])

A ideia agora é zerar todos os outros elementos da coluna $x_2$. Para tanto, fazemos as seguintes substituições:

* $L_2 \rightarrow L_2 - 4 L_1$
* $L_3 \rightarrow L_3 + 22 L_1$

In [6]:
for i in range(M.shape[0]):
    if(i != pivot_row):
        M[i] -= M[i][pivot_col] * M[pivot_row]
M

array([[  0. ,   1.6,   1. ,   0.8,   0.2,   0. ,  24. ],
       [  0. ,  -1.4,   0. ,  -1.2,  -0.8,   1. , 104. ],
       [  1. ,  15.2,   0. ,  -0.4,   4.4,   0. , 528. ]])

### **Iteração 2**

Agora, a coluna pivô é a coluna de $x_3$. Efetuamos a divisão *element-wise* entre a coluna $b$ e a coluna $x_3$ para obter a linha pivô:

In [7]:
pivot_col = 3
div_pivot_b_cols = (M.T[b_col] / M.T[pivot_col])[:-1]
div_pivot_b_cols

array([ 30.   , -86.667])

In [8]:
pivot_row = np.argmin(np.abs(div_pivot_b_cols))
pivot_row

np.int64(0)

A linha pivô agora é $L_1$. Fazemos, então, a substituição $L_1 \rightarrow -\frac{L_1}{0.8}$.

In [9]:
M[pivot_row] /= M[pivot_row][pivot_col]
M

array([[  0.  ,   2.  ,   1.25,   1.  ,   0.25,   0.  ,  30.  ],
       [  0.  ,  -1.4 ,   0.  ,  -1.2 ,  -0.8 ,   1.  , 104.  ],
       [  1.  ,  15.2 ,   0.  ,  -0.4 ,   4.4 ,   0.  , 528.  ]])

Para zerar os outros elementos da coluna $x_3$:

* $L_2 \rightarrow L_2 + 1.2 L_1$
* $L_3 \rightarrow L_3 + 0.4 L_1$

In [10]:
for i in range(M.shape[0]):
    if(i != pivot_row):
        M[i] -= M[i][pivot_col] * M[pivot_row]
M

array([[  0.  ,   2.  ,   1.25,   1.  ,   0.25,   0.  ,  30.  ],
       [  0.  ,   1.  ,   1.5 ,   0.  ,  -0.5 ,   1.  , 140.  ],
       [  1.  ,  16.  ,   0.5 ,   0.  ,   4.5 ,   0.  , 540.  ]])

Repare que a linha de coeficientes de custo agora tem apenas valores positivos! Isto significa que chegamos ao final das iterações.

### **Resultado final**

**Lucro da companhia**

Podemos ver, pelo resultado final, que a variável original que faz parte da base é apenas $x_3$, e que a resposta final é:

* $x_1 = 0$;
* $x_2 = 0$;
* $x_3 = 30$.

O ponto máximo da função é $Z = 540$.

Isto significa que devem ser produzidos 30 litros de Combustível C, e nenhuma quantidade dos outros combustíveis, o que levará a um lucro de $\textrm{R\$ } 540$.

**Folga de matéria prima**

Substituindo os valores de $\boldsymbol{x}$ nas restrições:

\begin{equation*}
\begin{aligned}
8 \cdot 0 + 5 \cdot 0 + 4 \cdot 30 = 120 \\
5 \cdot 0 + 4 \cdot 0 + 2 \cdot 30 = 60
\end{aligned}
\end{equation*}

Pelos resultados, concluímos que:

* Não sobra extrato mineral;
* Sobram $200 - 60 = 140$ litros de solvente.

## **3. Solução por algoritmo**

In [15]:
A = np.array([
    [8, 5, 4],
    [5, 4, 2]
])

b = [120, 200]

coef = np.array([20, 22, 18]) * (-1)

var_bounds = np.array([[0, 0, 0], [np.inf, np.inf, np.inf]]).T

res = linprog(coef, A_ub=A, b_ub=b, bounds=var_bounds, method='highs')

In [16]:
print(f"Valor máximo da função objetivo: {-res.fun}")
print(f"x1 = {res.x[0]}, x2 = {res.x[1]}, x3 = {res.x[2]}")

Valor máximo da função objetivo: 540.0
x1 = 0.0, x2 = 0.0, x3 = 30.0
